In [1]:
from kafka import KafkaProducer
import time 
import json

producer = KafkaProducer(bootstrap_servers='localhost:9092',
                         value_serializer=lambda v:
                         json.dumps(v).encode('utf-8')
)

# هنعمل توليد لرساله كل ثانيه تتبعت ل التوبيك الي هو spark-topic
for i in range(1,20):
    data ={"id":i,"value":f"message{i}"}
    producer.send('spark-topic',value=data)
    print(f"sent{data}")



producer.flush() # يتاكد ان كل البافرز اتبعت فعلا  
producer.close()# يقفل الاتصال نظيف 






sent{'id': 1, 'value': 'message1'}
sent{'id': 2, 'value': 'message2'}
sent{'id': 3, 'value': 'message3'}
sent{'id': 4, 'value': 'message4'}
sent{'id': 5, 'value': 'message5'}
sent{'id': 6, 'value': 'message6'}
sent{'id': 7, 'value': 'message7'}
sent{'id': 8, 'value': 'message8'}
sent{'id': 9, 'value': 'message9'}
sent{'id': 10, 'value': 'message10'}
sent{'id': 11, 'value': 'message11'}
sent{'id': 12, 'value': 'message12'}
sent{'id': 13, 'value': 'message13'}
sent{'id': 14, 'value': 'message14'}
sent{'id': 15, 'value': 'message15'}
sent{'id': 16, 'value': 'message16'}
sent{'id': 17, 'value': 'message17'}
sent{'id': 18, 'value': 'message18'}
sent{'id': 19, 'value': 'message19'}


In [2]:
# هنعمل بقا كونسيومر علي الداتا دي ونعمل عليها بروسسنج في سبارك 

from pyspark.sql import SparkSession
from pyspark.sql.functions import from_json,col
from pyspark.sql.types import StringType ,StructType ,IntegerType

spark = SparkSession.builder\
.appName("Kafkasparkexample")\
.getOrCreate()


#    schema عمليه بناء ال 
schema= StructType() \
    .add("id",IntegerType()) \
    .add("value",StringType())


#  قراءه الاستريم من كاكفا 
df= spark.readStream .format("kafka") \
    .option ("kafka.bootstrap.servers","localhost:9092")\
    .option("subscribe","spark-topic")\
    .option("startingoffsets","earliest")\
    .load()


#  من بايت ل جيسون value  هنعمل تحويل ال  
json_df= df.select (from_json(col("value").cast("string"), schema).alias("data")).select("data.*")


# هنعرض الداتا بقا في الكونسول 
query = json_df.writeStream \
.outputMode("append")\
.format("console")\
.start()


Using Spark's default log4j profile: org/apache/spark/log4j2-defaults.properties
25/11/12 00:36:21 WARN Utils: Your hostname, ahmed-refat-VirtualBox, resolves to a loopback address: 127.0.1.1; using 10.0.2.15 instead (on interface enp0s3)
25/11/12 00:36:21 WARN Utils: Set SPARK_LOCAL_IP if you need to bind to another address
:: loading settings :: url = jar:file:/opt/spark/jars/ivy-2.5.3.jar!/org/apache/ivy/core/settings/ivysettings.xml
Ivy Default Cache set to: /home/ahmed-refat/.ivy2.5.2/cache
The jars for the packages stored in: /home/ahmed-refat/.ivy2.5.2/jars
org.apache.spark#spark-sql-kafka-0-10_2.13 added as a dependency
:: resolving dependencies :: org.apache.spark#spark-submit-parent-ad719748-2c85-42a2-955e-c9326c42fa43;1.0
	confs: [default]
	found org.apache.spark#spark-sql-kafka-0-10_2.13;4.0.1 in central
	found org.apache.spark#spark-token-provider-kafka-0-10_2.13;4.0.1 in central
	found org.apache.kafka#kafka-clients;3.9.1 in central
	found org.lz4#lz4-java;1.8.0 in centra

-------------------------------------------
Batch: 0
-------------------------------------------
+---+---------+
| id|    value|
+---+---------+
|  1| message1|
|  2| message2|
|  3| message3|
|  4| message4|
|  5| message5|
|  6| message6|
|  7| message7|
|  8| message8|
|  9| message9|
| 10|message10|
| 11|message11|
| 12|message12|
| 13|message13|
| 14|message14|
| 15|message15|
| 16|message16|
| 17|message17|
| 18|message18|
| 19|message19|
|  1| message1|
+---+---------+
only showing top 20 rows


25/11/12 21:47:41 WARN KafkaOffsetReaderAdmin: Error in attempt 1 getting Kafka offsets: 
java.util.concurrent.ExecutionException: org.apache.kafka.common.errors.TimeoutException: Call(callName=describeTopics, deadlineMs=1762905829741, tries=1, nextAllowedTryMs=1762976861621) timed out at 1762976861534 after 1 attempt(s)
	at java.base/java.util.concurrent.CompletableFuture.reportGet(CompletableFuture.java:396)
	at java.base/java.util.concurrent.CompletableFuture.get(CompletableFuture.java:2073)
	at org.apache.kafka.common.internals.KafkaFutureImpl.get(KafkaFutureImpl.java:165)
	at org.apache.spark.sql.kafka010.ConsumerStrategy.retrieveAllPartitions(ConsumerStrategy.scala:65)
	at org.apache.spark.sql.kafka010.ConsumerStrategy.retrieveAllPartitions$(ConsumerStrategy.scala:64)
	at org.apache.spark.sql.kafka010.SubscribeStrategy.retrieveAllPartitions(ConsumerStrategy.scala:101)
	at org.apache.spark.sql.kafka010.SubscribeStrategy.assignedTopicPartitions(ConsumerStrategy.scala:112)
	at org.a